In [164]:
import numpy as np
import pandas as pd

from feast import FeatureStore

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C h

In [165]:
# 1. Connect to the feature store
store = FeatureStore("/media/gaurav/ubuntudata/SemThree/MLops/Assigment1/MLOps_Group_49/feature_store/housing_feature_repo/feature_repo")

# 2. Create the entity DataFrame
# This DataFrame contains the entities and timestamps for which we want features.
# It must also contain the target variable for training.
entity_df = pd.read_parquet("/media/gaurav/ubuntudata/SemThree/MLops/Assigment1/MLOps_Group_49/feature_store/housing_feature_repo/feature_repo/data/housing_features.parquet")[[
    "event_timestamp",
    "house_id",
    "MedHouseVal"
]]

# 3. Retrieve historical features to build the training dataset
print("Retrieving training data from Feast...")
training_data = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "location_features:MedInc",
        "location_features:HouseAge",
        "location_features:AveBedrms",
        "location_features:Latitude",
    ],
).to_df()

print("\n--- Training Data ---")
print(training_data.head())

# 4. Train a model
print("\n--- Training Model ---")

# Define features (X) and target (y)
X = training_data[['MedInc', 'HouseAge', 'AveBedrms', 'Latitude']]
y = training_data['MedHouseVal']

# Split data for training and testing



<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:488: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Retrieving training data from Feast...

--- Training Data ---
                   event_timestamp house_id  MedHouseVal  MedInc  HouseAge  \
0 2025-07-05 12:44:49.947909+00:00    11001      5.00001  9.3845      26.0   
1 2025-07-05 12:44:49.947909+00:00     5734      4.29100  5.1718      31.0   
2 2025-07-05 12:44:49.947909+00:00     1596      3.10300  5.2079      21.0   
3 2025-07-05 12:44:49.947909+00:00      604      1.95700  3.3021      36.0   
4 2025-07-05 12:44:49.947909+00:00    13888      0.64700  2.0000      15.0   

   AveBedrms  Latitude  
0   1.010724     33.75  
1   1.063985     34.16  
2   1.086522     37.89  
3   1.068522     37.69  
4   1.467078     34.85  

--- Training Model ---


In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [167]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from mlflow.tracking import MlflowClient
import yaml

# Replace these with your actual training data
# X_train, X_test, y_train, y_test must be defined before running this script

# Set MLflow experiment
mlflow.set_experiment("Model_Comparison_Regression_MLOPs")

best_overall_model = None
best_overall_r2 = float('-inf')
best_model_name = ""
best_run_id = ""
best_artifact_path = ""

# --- Linear Regression ---
with mlflow.start_run(run_name="Linear_Regression") as run_lr:
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_preds = lr_model.predict(X_test)

    mse_lr = float(mean_squared_error(y_test, lr_preds))
    r2_lr = float(r2_score(y_test, lr_preds))

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("mse", mse_lr)
    mlflow.log_metric("r2", r2_lr)

    artifact_path_lr = "linear_regression_model"
    mlflow.sklearn.log_model(
        sk_model=lr_model,
        artifact_path=artifact_path_lr,
        registered_model_name="LinearRegressionModel"
    )

    print("\nLinear Regression:")
    print(f"  MSE: {mse_lr:.4f}")
    print(f"  R²: {r2_lr:.4f}")

    if r2_lr > best_overall_r2:
        best_overall_model = lr_model
        best_overall_r2 = r2_lr
        best_model_name = "LinearRegressionModel"
        best_run_id = run_lr.info.run_id
        best_artifact_path = artifact_path_lr

# --- Decision Tree with Grid Search ---
with mlflow.start_run(run_name="Tuned_Decision_Tree") as run_dt:
    param_grid = {
        'max_depth': [3, 5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(
        estimator=DecisionTreeRegressor(random_state=42),
        param_grid=param_grid,
        scoring='r2',
        cv=5,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    best_dt_model = grid_search.best_estimator_
    dt_preds = best_dt_model.predict(X_test)

    mse_dt = float(mean_squared_error(y_test, dt_preds))
    r2_dt = float(r2_score(y_test, dt_preds))

    best_params = {
        k: (str(v) if v is None else v)
        for k, v in grid_search.best_params_.items()
    }

    mlflow.log_param("model_type", "DecisionTreeRegressor")
    mlflow.log_params(best_params)
    mlflow.log_metric("mse", mse_dt)
    mlflow.log_metric("r2", r2_dt)

    artifact_path_dt = "decision_tree_model"
    mlflow.sklearn.log_model(
        sk_model=best_dt_model,
        artifact_path=artifact_path_dt,
        registered_model_name="DecisionTreeRegressorModel"
    )

    print("\nTuned Decision Tree Regressor:")
    print(f"  Best Parameters: {best_params}")
    print(f"  MSE: {mse_dt:.4f}")
    print(f"  R²: {r2_dt:.4f}")

    if r2_dt > best_overall_r2:
        best_overall_model = best_dt_model
        best_overall_r2 = r2_dt
        best_model_name = "DecisionTreeRegressorModel"
        best_run_id = run_dt.info.run_id
        best_artifact_path = artifact_path_dt

# --- Export metrics safely to YAML ---
client = MlflowClient()

mse_history = client.get_metric_history(best_run_id, "mse")
r2_history = client.get_metric_history(best_run_id, "r2")

# Convert to primitive dictionaries
mse_safe = [{
    "key": m.key,
    "value": m.value,
    "step": m.step,
    "timestamp": m.timestamp,
    "run_id": m.run_id,
    "model_id": getattr(m, "model_id", None)
} for m in mse_history]

r2_safe = [{
    "key": m.key,
    "value": m.value,
    "step": m.step,
    "timestamp": m.timestamp,
    "run_id": m.run_id,
    "model_id": getattr(m, "model_id", None)
} for m in r2_history]

# Save to YAML
with open("best_model_metrics.yaml", "w") as f:
    yaml.dump({
        "best_model_name": best_model_name,
        "best_run_id": best_run_id,
        "mse_history": mse_safe,
        "r2_history": r2_safe
    }, f)


# --- Final Output ---
print(f"\n✅ Best Model: {best_model_name} with R² = {best_overall_r2:.4f}")
print(f"   Run ID: {best_run_id}")
print("   Metrics saved to: best_model_metrics.yaml")



2025/08/03 13:50:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/media/gaurav/ubuntudata/SemThree/MLops/mlopsenv/lib/python3.12/site-packages/mlflow/models/model.py:421: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.utc_time_created = str(utc_time_created or datetime.utcnow())
2025/08/03 13:51:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Linear Regression:
  MSE: 0.6366
  R²: 0.5197


Registered model 'LinearRegressionModel' already exists. Creating a new version of this model...
Created version '18' of model 'LinearRegressionModel'.
2025/08/03 13:51:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/media/gaurav/ubuntudata/SemThree/MLops/mlopsenv/lib/python3.12/site-packages/mlflow/models/model.py:421: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.utc_time_created = str(utc_time_created or datetime.utcnow())
2025/08/03 13:52:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Tuned Decision Tree Regressor:
  Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
  MSE: 0.5576
  R²: 0.5794

✅ Best Model: DecisionTreeRegressorModel with R² = 0.5794
   Run ID: 8dcd8da2dff24a2ba139d12041776e1c
   Metrics saved to: best_model_metrics.yaml


Registered model 'DecisionTreeRegressorModel' already exists. Creating a new version of this model...
Created version '17' of model 'DecisionTreeRegressorModel'.


In [168]:
# --- Register the best model ---
model_uri = f"runs:/{best_run_id}/{best_artifact_path}"
registration_result = mlflow.register_model(
    model_uri=model_uri,
    name="HousingPricePredication"
)


Registered model 'HousingPricePredication' already exists. Creating a new version of this model...
2025/08/03 13:52:10 WARNING mlflow.tracking._model_registry.fluent: Run with id 8dcd8da2dff24a2ba139d12041776e1c has no artifacts at artifact path 'decision_tree_model', registering model based on models:/m-727b00743cf043f78913022a013470e5 instead
Created version '4' of model 'HousingPricePredication'.


In [169]:
import pickle

with open("../models/model.pkl", "wb") as f:
    pickle.dump(best_dt_model, f)